In [3]:
%load_ext autoreload
%autoreload 2

import imageio
import jax
from jax import numpy as jnp
import numpy as np
from omegaconf import OmegaConf
from orbax import checkpoint as ocp
from waymax import datatypes
from waymax import visualization

from rl.config.config import Config
from rl.mappo import RunnerState, init_config
from rl.model import ActorRNN
from rl.wrappers.marl import WaymaxWrapper
from utils import batchify, init_or_restore_run
# !export CUDA_VISIBLE_DEVICES=1
!export JAX_PLATFORM_NAME='cpu'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/jupyter-earle/miniconda3/envs/waymax/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [4]:
# @hydra.main(version_base=None, config_path="config", config_name="mappo_homogenous_rnn_waymax")
# Manually load the hydra
config: Config = OmegaConf.create(Config())
config.SEED = 7
init_config(config)
config.NUM_ENVS = 1
rng = jax.random.PRNGKey(config.SEED)
options = ocp.CheckpointManagerOptions(
    max_to_keep=2, create=True)
checkpoint_manager = ocp.CheckpointManager(
    config.ckpt_dir, options=options)
runner_state: RunnerState
env: WaymaxWrapper
latest_update_step = checkpoint_manager.latest_step()
runner_state, env, scenario, latest_update_step, wandb_run_id, wandb_resume = \
    init_or_restore_run(config, checkpoint_manager, latest_update_step, rng)
actor_network = ActorRNN(env.action_space(env.agents[0]).shape[0], config=config)

2024-04-22 23:34:51.224103: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


--- TOTAL STARTUP COSTS (make data_iter + env) = 0.005413984879851341 s ---
 of which 

--- DATA ITER COSTS = 0.00459025613963604 s ---
--- NEXT DATA ITER COSTS = 0.004590487107634544 s ---


In [5]:
# actor = create_netty_actor(network=actor_network, params=runner_state.train_states[0].params,
#                             is_controlled_func=env.get_avail_actions)

import os
from utils import unbatchify


rng = jax.random.PRNGKey(0)
init_obs, init_state = env.reset(scenario, rng)
actor_params = runner_state.train_states[0].params
actor_hidden = runner_state.hstates[0]

def step_env(carry, _):
    rng, obs, state, done, actor_hidden = carry
    # print(obs.shape)

    # traj = datatypes.dynamic_index(
    #     state.env_state.sim_trajectory, state.env_state.timestep, axis=-1, keepdims=True
    # )
    avail_actions = env.get_avail_actions(state.env_state)
    avail_actions = jax.lax.stop_gradient(
        batchify(avail_actions, env.agents, len(env.agents))
    )
    ac_in = (
        obs[np.newaxis, :],
        # obs,
        done[np.newaxis, :],
        # done,
        avail_actions[np.newaxis, :],
    )
    actor_hidden, pi = actor_network.apply(actor_params, actor_hidden, ac_in)            
    action = pi.sample(seed=rng)
    env_act = unbatchify(
        action, env.agents, config.NUM_ENVS, env.num_agents
    )
    env_act = {k: v.squeeze() for k, v in env_act.items()}

    # outputs = [
    #     jit_select_action({}, state, obs, None, rng)
    #     for jit_select_action in jit_select_action_list
    # ]
    # action = agents.merge_actions(outputs)
    obs, next_state, reward, done, info = env.step(state=state, action=env_act, scenario=scenario, key=rng)
    rng, _ = jax.random.split(rng)
    done = batchify(done, env.agents, env.num_agents)[:, 0]
    obs = batchify(obs, env.agents, env.num_agents)

    return (rng, obs, next_state, done, actor_hidden), next_state

remaining_timesteps = init_state.env_state.remaining_timesteps
    
done = jnp.zeros((len(env.agents),), dtype=bool)
init_obs = batchify(init_obs, env.agents, env.num_agents)

_, states = jax.lax.scan(step_env, (rng, init_obs, init_state, done, actor_hidden), None, length=remaining_timesteps)

# states = jax.tree.map(lambda x, y: jnp.concatenate([x[None], y], axis=0), init_state, states)

# states = []
# rng, obs, state, done, actor_hidden = (rng, init_obs, init_state, done, actor_hidden)
# for i in range(remaining_timesteps):
#     carry, state = step_env((rng, obs, state, done, actor_hidden), None)
#     rng, obs, state, done, actor_hidden = carry
#     states.append(state)

# frames = [visualization.plot_simulator_state(init_state.env_state, use_log_traj=False)]
frames = []
for i in range(remaining_timesteps):
    if i % 5 == 0:
        # state = jax.tree.map(lambda x: x[i] if len(x.shape) > 0 else x, states)
        state = jax.tree.map(lambda x: x[i], states)
        # Print all leaf names
        frames.append(visualization.plot_simulator_state(state.env_state, use_log_traj=False))

imageio.mimsave(os.path.join(config.vid_dir, f"enjoy_{latest_update_step}.gif", frames, fps=10))

{'object_0': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_1': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_10': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_11': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_12': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_13': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_14': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_15': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_16': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_17': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_18': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_19': Traced<ShapedArray(float32[2])>with<DynamicJaxprTrace(level=2/0)>, 'object_2': Traced<ShapedArray(float32[2]

: 